# Problem Statement 1: 
- Build a graph-based representation of NYC zones (taxi zones adjacency graph)

In [1]:
import geopandas as gpd
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
taxi_zone = pd.read_csv(r"C:\Users\Dell\Downloads\taxi_zone_lookup.csv")

In [3]:
taxi_zone

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone
...,...,...,...,...
260,261,Manhattan,World Trade Center,Yellow Zone
261,262,Manhattan,Yorkville East,Yellow Zone
262,263,Manhattan,Yorkville West,Yellow Zone
263,264,Unknown,NaN,NaN


In [4]:
taxi_zone["Borough"].unique()

array(['EWR', 'Queens', 'Bronx', 'Manhattan', 'Staten Island', 'Brooklyn',
       'Unknown', nan], dtype=object)

In [5]:
zones = gpd.read_file(r"C:\Users\Dell\Downloads\taxi_zones")

In [6]:
zones

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((933100.918 192536.086, 933091.011 19..."
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"MULTIPOLYGON (((1033269.244 172126.008, 103343..."
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((1026308.77 256767.698, 1026495.593 2..."
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((992073.467 203714.076, 992068.667 20..."
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((935843.31 144283.336, 936046.565 144..."
...,...,...,...,...,...,...,...
258,259,0.126750,0.000395,Woodlawn/Wakefield,259,Bronx,"POLYGON ((1025414.782 270986.139, 1025138.624 ..."
259,260,0.133514,0.000422,Woodside,260,Queens,"POLYGON ((1011466.966 216463.005, 1011545.889 ..."
260,261,0.027120,0.000034,World Trade Center,261,Manhattan,"POLYGON ((980555.204 196138.486, 980570.792 19..."
261,262,0.049064,0.000122,Yorkville East,262,Manhattan,"MULTIPOLYGON (((999804.795 224498.527, 999824...."


# Using Plotly

In [7]:
zones_wgs = zones.to_crs(epsg=4326)

In [8]:
zones_wgs

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((-74.18445 40.695, -74.18449 40.6951,..."
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"MULTIPOLYGON (((-73.82338 40.63899, -73.82277 ..."
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((-73.84793 40.87134, -73.84725 40.870..."
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((-73.97177 40.72582, -73.97179 40.725..."
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((-74.17422 40.56257, -74.17349 40.562..."
...,...,...,...,...,...,...,...
258,259,0.126750,0.000395,Woodlawn/Wakefield,259,Bronx,"POLYGON ((-73.85107 40.91037, -73.85207 40.909..."
259,260,0.133514,0.000422,Woodside,260,Queens,"POLYGON ((-73.90175 40.76078, -73.90147 40.759..."
260,261,0.027120,0.000034,World Trade Center,261,Manhattan,"POLYGON ((-74.01333 40.70503, -74.01327 40.704..."
261,262,0.049064,0.000122,Yorkville East,262,Manhattan,"MULTIPOLYGON (((-73.94383 40.78286, -73.94376 ..."


In [9]:
nodes = zones_wgs[['LocationID', 'zone', 'borough', 'geometry']].copy()
nodes['lon'] = nodes.geometry.centroid.x
nodes['lat'] = nodes.geometry.centroid.y

C:\Users\Dell\AppData\Local\Temp\ipykernel_11316\11758070.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  nodes['lon'] = nodes.geometry.centroid.x
C:\Users\Dell\AppData\Local\Temp\ipykernel_11316\11758070.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  nodes['lat'] = nodes.geometry.centroid.y


In [10]:

G = nx.Graph()

In [11]:

for idx, row in nodes.iterrows():
    G.add_node(row.LocationID)

In [12]:

for i, row1 in nodes.iterrows():
    for j, row2 in nodes.iterrows():
        if i < j:  
            if row1.geometry.touches(row2.geometry):
                G.add_edge(row1.LocationID, row2.LocationID)

In [13]:

positions = {row.LocationID: (row.lon, row.lat) for idx, row in nodes.iterrows()}


edge_x = []
edge_y = []

for u, v in G.edges():
    x0, y0 = positions[u]
    x1, y1 = positions[v]
    edge_x += [x0, x1, None]
    edge_y += [y0, y1, None]

In [14]:
import plotly.io as pio
pio.renderers.default = "browser"   

import plotly.graph_objects as go


borough_colors_plotly = {
    "Manhattan": "blue",
    "Brooklyn": "green",
    "Queens": "orange",
    "Bronx": "red",
    "Staten Island": "purple",
    "EWR": "black"
}


nodes['color'] = nodes['borough'].map(borough_colors_plotly)

edge_trace = go.Scatter(
    x=edge_x,
    y=edge_y,
    mode='lines',
    line=dict(
        width=1.5,     
        color='black',  
    ),
    hoverinfo='none',
    name='Adjacency (touching zones)'
)


node_trace = go.Scatter(
    x=nodes['lon'],
    y=nodes['lat'],
    mode='markers',
    marker=dict(
        size=7,
        color=nodes['color'],
        line=dict(width=0.5, color='black')  
    ),
    customdata=nodes[['LocationID', 'zone', 'borough']].values,
    hovertemplate=(
        "<b>LocationID:</b> %{customdata[0]}<br>"
        "<b>Zone:</b> %{customdata[1]}<br>"
        "<b>Borough:</b> %{customdata[2]}<extra></extra>"
    ),
    name='Taxi zones'
)


fig = go.Figure(data=[edge_trace, node_trace])


for borough, color in borough_colors_plotly.items():
    fig.add_trace(
        go.Scatter(
            x=[None],
            y=[None],
            mode='markers',
            marker=dict(size=10, color=color),
            legendgroup=borough,
            showlegend=True,
            name=borough
        )
    )

fig.update_layout(
    title="NYC Taxi Zones Adjacency Graph (Interactive)",
    showlegend=True,
    legend=dict(
        x=0.01,
        y=0.99,
        bgcolor="rgba(255,255,255,0.7)"
    ),
    yaxis=dict(scaleanchor="x", scaleratio=1),  
)

fig.show()
